#### Ex01-4 Minimizing free time of resources + late time cost with early time benefit (scoring by priority)

In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp
from datetime import datetime, timedelta

In [2]:
df_manpowers = pd.read_excel("scheduling.xlsx", sheet_name="Manpowers")
df_customer_orders = pd.read_excel("scheduling.xlsx", sheet_name="Customer_mockup")

df_customer_orders["Schedule Delivery"] = df_customer_orders["Schedule Delivery"].astype(str)

manpower_list = df_manpowers["Name"].to_list()
order_list = df_customer_orders["Order"].to_list()

max_manpower_possible = len(df_manpowers)
regular_work_mins = 8 * 60

early_cost_weight = 1
late_costs_weight = 10

In [3]:
# Define calendar
start_date = datetime.strptime(
    df_customer_orders["Schedule Delivery"].min().split(" ")[0], "%Y-%m-%d"
)
end_date = datetime.strptime(
    df_customer_orders["Schedule Delivery"].max().split(" ")[0], "%Y-%m-%d"
)

date_modified = start_date
calendar_list = [start_date.strftime("%Y-%m-%d")]

while date_modified < end_date:
    date_modified += timedelta(days=1)
    calendar_list.append(date_modified.strftime("%Y-%m-%d"))

In [4]:
def get_date_object_from_attr(df, attr):
    object_attr = {}
    for date in calendar_list:
        for order in order_list:
            try:
                object_attr[(date, order)] = df[attr][
                    (df["Order"] == order)
                    & (df["Schedule Delivery"] == date)
                    ].item()
            except ValueError:
                object_attr[(date, order)] = 0
    return object_attr

def get_object_from_attr(df, attr):
    object_attr = {}
    for order in order_list:
        try:
            object_attr[(order)] = df[attr][
                (df["Order"] == order)
                ].item()
        except ValueError:
            object_attr[(order)] = 0
    return object_attr

In [5]:
# daily_requirements = cycle_times * amounts
daily_requirements = {}
amount_by_dates = get_date_object_from_attr(df_customer_orders, "Amount")
cycle_time_by_dates = get_date_object_from_attr(df_customer_orders, "Cycle Time (Min)")
for key in cycle_time_by_dates:
    daily_requirements[key] = cycle_time_by_dates[key] * amount_by_dates[key]
    
priority_scores = get_object_from_attr(df_customer_orders, "Priority Score")
capacity_manpowers = get_object_from_attr(df_customer_orders, "Manpower Capacity")
delivery_dates = get_object_from_attr(df_customer_orders, "Schedule Delivery")
amounts = get_object_from_attr(df_customer_orders, "Amount")
cycle_times = get_object_from_attr(df_customer_orders, "Cycle Time (Min)")
order_requirements = {}
for key in cycle_times:
    order_requirements[key] = cycle_times[key] * amounts[key]

In [6]:
model = gp.Model('Scheduling')

# Manpower Working Status in each date of each order
gp_manpower_status = model.addVars(calendar_list, manpower_list, order_list, vtype=gp.GRB.BINARY, name="ManpowerStatus")
# Number of manpowers should be less than or equal to the capacity
model.addConstrs(gp_manpower_status.sum(date, "*", order) <= capacity_manpowers[(order)]
                for date in calendar_list for order in order_list)

# Order Status based on Manpower Working Status (0 for no order working, 1 for order working)
gp_order_status = model.addVars(calendar_list, order_list, vtype=gp.GRB.BINARY, name="OrderStatus")
# Order Status should be 1 if any manpower working on that order
model.addConstrs(gp_order_status[(date, order)] <= gp_manpower_status.sum(date, "*", order)
                for date in calendar_list for order in order_list)

# Work Time Balance for any manpower
gp_balance_worktime = model.addVars(order_list, vtype=gp.GRB.CONTINUOUS, name="BalanceWorktime")
# Work Time Balance for any order is calculated by cycle time / summing up all manpowers status working on that order
model.addConstrs(gp_balance_worktime[(order)] * gp_manpower_status.sum("*", "*", order) == order_requirements[(order)]
                for order in order_list)

# Work Time for each order given the manpower and date
gp_manpower_worktime = model.addVars(calendar_list, manpower_list, order_list, vtype=gp.GRB.CONTINUOUS, name="ManpowerWorktime")
# Work Time for each manpower should be equal to the balance work time but considered with manpower status
model.addConstrs(gp_manpower_worktime[(date, manpower, order)] == gp_manpower_status[(date, manpower, order)] * gp_balance_worktime[(order)]
                for date in calendar_list for manpower in manpower_list for order in order_list)
# Total work time of all orders for each of manpower in each day should not exceed the regular work time
model.addConstrs(gp_manpower_worktime.sum(date, manpower, "*") <= regular_work_mins
                for date in calendar_list for manpower in manpower_list)

# Free Time for each manpower in each day
gp_manpower_freetime = model.addVars(calendar_list, manpower_list, vtype=gp.GRB.CONTINUOUS, name="ManpowerFreetime")
# Free Time for each manpower should be equal to the regular work time minus the work time
model.addConstrs(gp_manpower_freetime[(date, manpower)] == regular_work_mins - gp_manpower_worktime.sum(date, manpower, "*")
                for date in calendar_list for manpower in manpower_list)

# Total work time = all required work time
model.addConstr(gp.quicksum(gp_manpower_worktime[(date, manpower, order)] for date in calendar_list for manpower in manpower_list for order in order_list) 
                == (gp.quicksum(daily_requirements[(date, order)] for date in calendar_list for order in order_list)))

# The same order should be done in the same day
model.addConstrs(gp_manpower_worktime.sum(date, "*", order) == order_requirements[(order)] * gp_order_status[(date, order)]
                for date in calendar_list for order in order_list)

# Variable gap early/late work time
gp_gap_worktime = model.addVars(calendar_list, order_list, lb=-1e6, ub=1e6, vtype=gp.GRB.CONTINUOUS, name="GapWorkTime")
abs_gp_gap_worktime = model.addVars(calendar_list, order_list, vtype=gp.GRB.CONTINUOUS, name="AbsGapWorkTime")

# Set the value of gap worktime (positive for early, negative for late)
for l in range(len(calendar_list)):
    model.addConstrs(gp_gap_worktime[(calendar_list[l], order)] == gp.quicksum(gp_manpower_worktime[(date, manpower, order)] for date in calendar_list[: l + 1] for manpower in manpower_list)
                                                                    - (gp.quicksum(daily_requirements[(date, order)] for date in calendar_list[: l + 1])) for order in order_list)
# Set the value of ABS(gap worktime)
model.addConstrs(abs_gp_gap_worktime[(date, order)] == gp.abs_(gp_gap_worktime[(date, order)])
                for date in calendar_list for order in order_list)
    
# Create variable "early worktime" and "early benefits"
early_worktime = model.addVars(calendar_list, order_list, vtype=gp.GRB.CONTINUOUS, name="EarlyWorkTime")
early_benefit = model.addVars(calendar_list, order_list, vtype=gp.GRB.CONTINUOUS, name="EarlyBenefits")
# Set the value of early worktime
model.addConstrs(early_worktime[(date, order)] == (gp_gap_worktime[(date, order)] + abs_gp_gap_worktime[(date, order)]) / 2 
                for date in calendar_list for order in order_list)
# Set the value of early benefits
model.addConstrs(early_benefit[(date, order)] == early_worktime[(date, order)] * priority_scores[order]
                for date in calendar_list for order in order_list)

# Create variable "late worktime" and "late costs"
late_worktime = model.addVars(calendar_list, order_list, vtype=gp.GRB.CONTINUOUS, name="LateWorkTime")
late_costs = model.addVars(calendar_list, order_list, vtype=gp.GRB.CONTINUOUS, name="LateCosts")
# Set the value of late worktime
model.addConstrs(late_worktime[(date, order)] == (abs_gp_gap_worktime[(date, order)] - gp_gap_worktime[(date, order)]) / 2
                for date in calendar_list for order in order_list)
# Set the value of late costs
model.addConstrs(late_costs[(date, order)] == late_worktime[(date, order)] * priority_scores[order]
                for date in calendar_list for order in order_list)

# Minimizing free time of resources
model.ModelSense = gp.GRB.MINIMIZE

objective = 0
objective += gp.quicksum(gp_manpower_freetime[(date, manpower)] for date in calendar_list for manpower in manpower_list)
objective -= gp.quicksum(early_cost_weight * early_benefit[(date, mo)] for date in calendar_list for mo in order_list)
objective += gp.quicksum(late_costs_weight * late_costs[(date, mo)] for date in calendar_list for mo in order_list)

model.setObjective(objective)
model.optimize()

sol = pd.DataFrame(data={"Solution": model.X}, index=model.VarName)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-05-03
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 433 rows, 948 columns and 3096 nonzeros
Model fingerprint: 0x985e2897
Model has 300 quadratic constraints
Model has 48 general constraints
Variable types: 612 continuous, 336 integer (336 binary)
Coefficient statistics:
  Matrix range     [5e-01, 6e+02]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+06]
  RHS range        [4e+00, 7e+03]
  QRHS range       [6e+02, 6e+02]
Presolve removed 206 rows and 174 columns
Presolve time: 0.01s
Presolved: 1103 rows, 1638 columns, 4238 nonzeros
Presolved model has 632 SOS constraint(s)
Variable types: 954 continuous, 684 integer (682 binary)

Root relaxation: objective -3.732000e+04, 712 iterations, 0.01 seconds (0.01 work uni

In [7]:
rows = calendar_list.copy()
columns = manpower_list.copy()
worktime_obj = {}

manpower_plan_by_order = pd.DataFrame(columns=columns, index=rows, data="")

for year, mine, order in gp_manpower_worktime.keys():
    try:
        worktime_obj[order].loc[year, mine] = np.round(gp_manpower_worktime[year, mine, order].x, 2)
        if worktime_obj[order].loc[year, mine] > 0:
            manpower_plan_by_order.loc[year, mine] = manpower_plan_by_order.loc[year, mine] + f" {order}={np.round(gp_manpower_worktime[year, mine, order].x, 2)}"
    except:
        worktime_obj[order] = pd.DataFrame(columns=columns, index=rows, data=0.0)
        worktime_obj[order].loc[year, mine] = np.round(gp_manpower_worktime[year, mine, order].x, 2)
        if worktime_obj[order].loc[year, mine] > 0:
            manpower_plan_by_order.loc[year, mine] = manpower_plan_by_order.loc[year, mine] + f" {order}={np.round(gp_manpower_worktime[year, mine, order].x, 2)}"
    
manpower_plan_by_order

,Mr. A,Mr. B,Mr. C,Mr. D,Mr. E,Mr. F
2021-11-01,A01=150.0 B05=150.0 B09=150.0,A02=150.0 B05=150.0 B09=150.0,A01=150.0 A02=150.0,A02=150.0 B05=150.0 B09=150.0,A01=150.0 A02=150.0 B05=150.0,A01=150.0 B09=150.0
2021-11-02,B01=150.0 B02=150.0 B04=150.0,B02=150.0 B03=150.0 B04=150.0,B01=150.0 B03=150.0 B04=150.0,B01=150.0 B02=150.0 B04=150.0,B02=150.0 B03=150.0,B01=150.0 B03=150.0
2021-11-03,B06=150.0 B10=150.0,B06=150.0 B08=150.0 B10=150.0,B06=150.0 B08=150.0 B10=150.0,B06=150.0 B08=150.0,B07=300.0 B08=150.0,B07=300.0 B10=150.0
2021-11-04,,,,,,
